In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV   #Performing grid search
from scipy.stats import skew
from collections import OrderedDict
from sklearn.inspection import permutation_importance
import shap
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import joblib
import numbers
import torch
from torch_geometric.loader import DataLoader  

In [ ]:
# plt.style.use("~/style.mplstyle")

In [ ]:
def set_seed(seed):
    import random
    random.seed(seed)  
    np.random.seed(seed) 
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False 

In [ ]:
def save_model_with_joblib(model, filename):
    joblib.dump(model, filename)

In [11]:
import torch
# 加载数据集
def load_data_list(file_path):
    """
    从指定文件路径加载数据列表。

    参数：
    file_path (str): 数据列表文件的路径。

    返回：
    list: 加载的数据列表。
    """
    return torch.load(file_path)

def data_loader(property, N=None):
    if not N:
        property_load_file_path = f'experiment_dataset/{property}_data_list.pt' # 总数据集
        property_load_train_dataset_path = f'experiment_dataset/train_{property}_data_list.pt' # 训练集
        property_load_val_dataset_path = f'experiment_dataset/val_{property}_data_list.pt' # 验证集
        property_load_test_dataset_path = f'experiment_dataset/test_{property}_data_list.pt' # 测试集
    else:
        property_load_file_path = f'experiment_dataset_N={N}/{property}_data_list.pt' # 总数据集
        property_load_train_dataset_path = f'experiment_dataset_N={N}/train_{property}_data_list.pt' # 训练集
        property_load_val_dataset_path = f'experiment_dataset_N={N}/val_{property}_data_list.pt' # 验证集
        property_load_test_dataset_path = f'experiment_dataset_N={N}/test_{property}_data_list.pt' # 测试集

    # 加载数据集
    property_loaded_data_list = load_data_list(property_load_file_path)
    property_loaded_train_data_list = load_data_list(property_load_train_dataset_path)
    property_loaded_val_data_list = load_data_list(property_load_val_dataset_path)
    property_loaded_test_data_list = load_data_list(property_load_test_dataset_path)

    return property_loaded_train_data_list, property_loaded_val_data_list, property_loaded_test_data_list

In [12]:
import xgboost as xgb
import optuna
import time
import torch
import numpy as np
from sklearn.metrics import mean_squared_error

def extract_features_targets(data_list):
    """
    从数据列表中提取moldescriptor作为特征X和y中指定索引的值作为目标Y。
    
    Args:
        data_list (list): 包含Data对象的列表。
        feature (str): 可选"2Ddescriptors"或“fp”，代表训练模型时使用的特征
        
    Returns:
        X (np.ndarray): 特征矩阵。
        Y (np.ndarray): 目标向量。
    """
    X = []
    Y = []
    for data in data_list:
        fp = data.morgan_fp.numpy().flatten() # 采用分子描述符作为特征
        X.append(fp)
        # 提取目标y值
        target = data.y.numpy().flatten()
        Y.append(target)
        
    
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

In [ ]:
from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNRegressor
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn

def batch_train_tabpfn_model(target_property_col_list, N=None, seed=42):
    """
    针对每个 (property, model) 组合训练并评估，返回测试集指标汇总 DataFrame。

    参数：
        df (pd.DataFrame): 原始数据（若需要，可在函数内加载）。
        target_property_col_list (List[str]): 需要预测的性质名称列表。
        model_type_list (List[str]): 模型类型列表，如 ["GAT","GCN","MLP","PolyBERT"]。
        epochs (int): 训练轮数。
        seed (int): 随机种子。

    返回:
        pd.DataFrame: 包含列 [property, model_type, test_mse, test_mae, test_r2]。
    """
    
    # 设置随机种子
    set_seed(seed)
    
    # 用于汇总所有结果的列表
    results = []

    for property in target_property_col_list:
        print(f"→ Training XGBoost on property: {property}")
        try:
            # 加载数据
            train_list, val_list, test_list = data_loader(property, N)

            # 提取特征和目标
            X_train, y_train = extract_features_targets(train_list)
            X_val, y_val = extract_features_targets(val_list)
            X_test, y_test = extract_features_targets(test_list)

            reg = AutoTabPFNRegressor(max_time=30, ignore_pretraining_limits=True) # runs for 30 seconds

            reg.fit(X_train, y_train)
            preds = reg.predict(X_test)

            property_test_mae = sklearn.metrics.mean_absolute_error(y_test, preds)
            property_test_mse = sklearn.metrics.mean_squared_error(y_test, preds)
            property_test_r2 = sklearn.metrics.r2_score(y_test, preds)
            
        except Exception as e:
            # 出错时跳过，并把指标置为 NaN
            print(f"⚠️  TabPFN failed on {property}: {e}")
            test_mse = test_mae = test_r2 = np.nan
        
        # 将结果追加到列表
        results.append({
            "property": property,
            "model_type": "TabPFN",
            "test_mse": property_test_mse,
            "test_mae": property_test_mae,
            "test_r2": property_test_r2
        })

    # 构造并返回 DataFrame
    return pd.DataFrame(results)


In [14]:
# 这里（N2和N4）需要重新跑！之前的代码用错了训练集了
target_property_col_list = ['Bandgap', 'CO2_Permeability',
       'Compressive_Strength', 'Crystallization_Temperature',
       'Elongation_at_Break', 'Flexural_Strength', 'Tg', 'H2_Permeability',
       'Hardness', 'Impact_Strength', 'Ion_Exchange_Capacity',
       'Limiting_Oxygen_Index', 'Lower_Critical_Solution_Temperature', 'Tm',
       'Methanol_Permeability', 'O2_Permeability', 'Refractive_Index',
       'Swelling_Degree', 'Thermal_Conductivity', 'Tensile_Strength', 'Td',
       'Upper_Critical_Solution_Temperature', 'Water_Contact_Angle',
       'Water_Uptake', 'Youngs_Modulus', 'Dielectric_Constant_Total'
       ]


In [15]:
df_tabpfn_test_performance_summary = batch_train_tabpfn_model(target_property_col_list, seed=42)
df_tabpfn_test_performance_summary.to_csv("tabpfn_test_performance_summary.csv",index=None)

2025-04-22 11:03:05 INFO     Using `default` preset for Post Hoc Ensemble.
2025-04-22 11:03:05 INFO     No categorical_feature_indices given. Assuming no categorical features.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
2025-04-22 11:03:05 INFO     Using task type: TaskType.REGRESSION
2025-04-22 11:03:05 INFO     Obtaining TabPFN models from a random portfolio.


→ Training XGBoost on property: Bandgap


2025-04-22 11:03:10 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

→ Training XGBoost on property: CO2_Permeability


2025-04-22 11:04:27 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

...

→ Training XGBoost on property: Water_Uptake


2025-04-22 11:36:44 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

→ Training XGBoost on property: Youngs_Modulus


2025-04-22 11:38:31 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

→ Training XGBoost on property: Dielectric_Constant_Total
⚠️  XGBoost failed on Dielectric_Constant_Total: [Errno 2] No such file or directory: 'experiment_dataset/Dielectric_Constant_Total_data_list.pt'


In [16]:
df_tabpfn_test_performance_summary = batch_train_tabpfn_model(target_property_col_list, N=2, seed=42)
df_tabpfn_test_performance_summary.to_csv("tabpfn_test_performance_summary_N=2.csv",index=None)

2025-04-22 11:38:57 INFO     Using `default` preset for Post Hoc Ensemble.
2025-04-22 11:38:57 INFO     No categorical_feature_indices given. Assuming no categorical features.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
2025-04-22 11:38:57 INFO     Using task type: TaskType.REGRESSION
2025-04-22 11:38:57 INFO     Obtaining TabPFN models from a random portfolio.


→ Training XGBoost on property: Bandgap


2025-04-22 11:39:01 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

2025-04-22 11:39:16 INFO     Likely not enough time left for another model.
2025-04-22 11:39:16 INFO     Stop validation of all models after 1 models in repeat 1.
2025-04-22 11:39:16 INFO     As this is the first repeat, we trim down the models to all so-far run models!
2025-04-22 11:39:16 INFO     Order of selections: [0]
2025-04-22 11:39:16 INFO     Val loss over iterations: [1.0398460635549116]
2025-04-22 11:39:16 INFO     Model losses: [1.03984606]
2025-04-22 11:39:16 INFO     Best weights: [1.]
Number of features 2048 is greater than the maximum Number of features 500 supported by the model. You may see degraded performance.
2025-04-22 11:39:32 INFO     Using `default` preset for Post Hoc Ensemble.
2025-04-22 11:39:32 INFO     No categorical_feature_indices given. Assuming no categorical features.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
2025-04-22 11:39:32 INFO     Using task type: TaskTyp

→ Training XGBoost on property: CO2_Permeability


...

2025-04-22 12:05:34 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

→ Training XGBoost on property: Youngs_Modulus


2025-04-22 12:06:03 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

→ Training XGBoost on property: Dielectric_Constant_Total


2025-04-22 12:06:39 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

In [17]:
df_tabpfn_test_performance_summary = batch_train_tabpfn_model(target_property_col_list, N=4, seed=42)
df_tabpfn_test_performance_summary.to_csv("tabpfn_test_performance_summary_N=4.csv",index=None)

2025-04-22 12:07:52 INFO     Using `default` preset for Post Hoc Ensemble.
2025-04-22 12:07:52 INFO     No categorical_feature_indices given. Assuming no categorical features.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
2025-04-22 12:07:52 INFO     Using task type: TaskType.REGRESSION
2025-04-22 12:07:52 INFO     Obtaining TabPFN models from a random portfolio.


→ Training XGBoost on property: Bandgap


2025-04-22 12:07:55 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

2025-04-22 12:08:13 INFO     Likely not enough time left for another model.
2025-04-22 12:08:13 INFO     Stop validation of all models after 1 models in repeat 1.
2025-04-22 12:08:13 INFO     As this is the first repeat, we trim down the models to all so-far run models!
2025-04-22 12:08:13 INFO     Order of selections: [0]
2025-04-22 12:08:13 INFO     Val loss over iterations: [1.0300592130837294]
2025-04-22 12:08:13 INFO     Model losses: [1.03005921]
2025-04-22 12:08:13 INFO     Best weights: [1.]
Number of features 2048 is greater than the maximum Number of features 500 supported by the model. You may see degraded performance.
2025-04-22 12:08:32 INFO     Using `default` preset for Post Hoc Ensemble.
2025-04-22 12:08:32 INFO     No categorical_feature_indices given. Assuming no categorical features.
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
2025-04-22 12:08:32 INFO     Using task type: TaskTyp

→ Training XGBoost on property: CO2_Permeability


...

2025-04-22 12:38:09 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

→ Training XGBoost on property: Youngs_Modulus


2025-04-22 12:38:41 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model

→ Training XGBoost on property: Dielectric_Constant_Total


2025-04-22 12:39:28 INFO     Using 100 base models: ['default_tabpfn_model_0', 'random_tabpfn_model_1', 'random_rf_pfn_model_2', 'random_tabpfn_model_3', 'random_rf_pfn_model_4', 'random_rf_pfn_model_5', 'random_rf_pfn_model_6', 'random_rf_pfn_model_7', 'random_tabpfn_model_8', 'random_tabpfn_model_9', 'random_rf_pfn_model_10', 'random_rf_pfn_model_11', 'random_tabpfn_model_12', 'random_rf_pfn_model_13', 'random_tabpfn_model_14', 'random_tabpfn_model_15', 'random_rf_pfn_model_16', 'random_tabpfn_model_17', 'random_tabpfn_model_18', 'random_rf_pfn_model_19', 'random_rf_pfn_model_20', 'random_rf_pfn_model_21', 'random_tabpfn_model_22', 'random_rf_pfn_model_23', 'random_tabpfn_model_24', 'random_tabpfn_model_25', 'random_rf_pfn_model_26', 'random_tabpfn_model_27', 'random_tabpfn_model_28', 'random_rf_pfn_model_29', 'random_tabpfn_model_30', 'random_rf_pfn_model_31', 'random_tabpfn_model_32', 'random_tabpfn_model_33', 'random_rf_pfn_model_34', 'random_tabpfn_model_35', 'random_rf_pfn_model